In [ ]:
import argparse
from PIL import Image
import numpy as np 
import os
import random
from collections import Counter
from itertools import islice
import torch


c:\Users\hannegga\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

a = torch.ones((4,3))
print(a)
b = torch.ones((2,4,4,3))
print(b) 
print(b.dim())
print(b.size())


tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
tensor([[[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],

         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],

         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],

         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]],


        [[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],

         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],

         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],

         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]]])
4
torch.Size([2, 4, 4, 3])


In [ ]:
a = torch.tensor([1, 1, 1])
b = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8, 9]).reshape((3, 3))

print(b)
print('A matrix multiplication')
#print(torch.matmul(b, a))

print()
print('Element-wise multiplication - what happened here?')
print(torch.mul(b, a))

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])
A matrix multiplication

Element-wise multiplication - what happened here?
tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])


In [17]:
#Computing gradients 

x = torch.tensor(data=[0., 2.], requires_grad=True)
y = torch.sum(torch.pow(x, 2))

print('x:', x)
print('y:', y)

# Run backwards propagation to compute the gradients in the graph
y.backward()

# Print the gradient
print(x.grad)

x: tensor([0., 2.], requires_grad=True)
y: tensor(4., grad_fn=<SumBackward0>)
tensor([0., 4.])


In [2]:
def get_training_and_test_set(train_path): 
    """
    Gets a path for where the training data is, and extracts the names of the images and the corresponding ground truth value. 
    These are shuffled to not get biased data when splitting the data up in a training set and a validation set. 
    splits the training data into 80% training and 20% validation. 

    Inputs: 
        train_path: relative path of where the training data is 
    Output: 
        validation_data: dictionary with data used for validation, giving the gorund truth value as the key and the 
        corresponding image names as a list of every image which has the ground truth value. 
        training_data: dictionary with data used for training, built up in the same way as validation_data
        training_test_data: dictionary with image as key and 0 as value.
    """
    relative_path = os.path.abspath(os.path.dirname('__file__'))
    train_path = os.path.join(relative_path, train_path)
    validation_data, training_test_data, training_data = {}, {}, {}
    with open(train_path + '\\truth.dsv', "r") as csv_file:
        data = csv_file.readlines()
    random.shuffle(data)
    i = 0 
    for pic in data: 
        i += 1
        if (i % 8 == 0): 
            validation_data[pic.split(':')[0]] = pic.split(':')[1]
      
        else: 
            if(not pic.split(':')[1].strip('\n') in training_data): 
                training_data[pic.split(':')[1].strip('\n')] = []
            training_data[pic.split(':')[1].strip('\n')].append(pic.split(':')[0])
            training_test_data[pic.split(':')[0]] = pic.split(':')[1].strip('\n')
    return training_data, validation_data, training_test_data

In [3]:
from PIL import Image
import torch 
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.nn as nn 
import torch.nn.functional as F 
import torchvision
import torchvision.transforms as transforms 
import matplotlib.pyplot as plt 
import numpy as np 
import random
from torch.utils.data import TensorDataset, DataLoader

class TensorData: 
    def __init__(self, label, image_name, image_tensor): 
        self.label = label 
        self.image_name = image_name 
        self.image_tensor = image_tensor
    
        


        

c:\Users\hannegga\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def get_training_list_as_tensor(path, test_set):
    picture = 0 
    for image in test_set: 
        picture = Image.open(path + "\\" + image)
        break
    image_tensor = torch.ones((len(test_set), 1,  picture.width, picture.height))   
    ground_truths = [] 
    i = 0 
    for pic_names, gt in test_set.items(): 
        picture = Image.open(path + "\\" + image)
        transform = transforms.Compose([transforms.PILToTensor()])
        ground_truths.append(gt)
        image_tensor[i] = transform(picture)
        i += 1
    return image_tensor, ground_truths
        
   

def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

input_size = 784 #28x28
hidden_size = 100
num_classes = 10 


num_epochs = 4
batch_size = 100 
learning_rate = 0.001


train_data_dict, validation_data_dict, test_data_dict = get_training_and_test_set('train_1000_28')
train_images, train_gts = get_training_list_as_tensor('train_1000_28', test_data_dict)
test_images, test_gts = get_training_list_as_tensor('train_1000_28', validation_data_dict)


my_dataset = TensorDataset(train_images)
#TODO se hvordan du må dele disse inn i batcher 
classes = list(train_data_dict.keys())




class ConvNet(torch.nn.Module): 
    def __init__(self): 
        super().__init__()
        #tror vi kun har 1 color channel, så input size er 1? 
        self.conv1 = nn.Conv2d(1, 8, kernel_size=4, padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(8, 20, kernel_size=4, padding=1)
        self.fc1 = nn.Linear(20*8*8, 120) #mulig det skal være 21*9*9
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(120, len(classes))

    def forward(self, x): 
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 20*9*9)
        x = F.relu(self.fc1())
        x = F.relu(self.fc2())
        x = self.fc3(x)
        return x 
        
    
model = ConvNet()
#crossentropyloss includes softmax. 
criterion = nn.CrossEntropyLoss()
#brukte SGD i videoen 
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
n_total_steps = len(test_data_dict)


for epoch in range(num_epochs): 
    
    images = training_image_list[epoch]
    labels = training_gt_list[epoch]
    for image, label in zip (images, labels): 
    
        #forward pass
        outputs = model(image)
        loss = criterion(outputs, label)
        
        #backward and optimize 
        optimizer.zero_grad()
        loss.backward() 
        optimizer.step() 
        
print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]

    images = test_image_list
    labels = test_gt_list
    outputs = model(images)
    # max returns (value ,index)
    _, predicted = torch.max(outputs, 1)
    n_samples += labels.size(0)
    n_correct += (predicted == labels).sum().item()
    
    for i in range(batch_size):
        label = labels[i]
        pred = predicted[i]
        if (label == pred):
            n_class_correct[label] += 1
        n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')
    
    
    
    




#torch_training_set = 

#test_dataset = torchvision.datasets.MNIST(root='./data', train = False, 
#                                          transform=transforms.ToTensor(), 
#                                          download=False)
#train_dataset = torchvision.datasets.MNIST(root='./data', train = True, transform=transforms.ToTensor(), download=True)
#train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
#test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = False)

#examples = iter(train_loader)
#samples, labels = examples.next()
#print(samples.shape, labels.shape)


